# *Radar plots* para comparación de deportistas

En este notebook vamos a visualizar una comparación entre varios deportistas en varios aspectos al mismo tiempo, de modo que queden visibles las fortalezas y debilidades de cada uno en un único vistazo. 

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None # Para deshabilitar algunos warnings
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "iframe"

Usamos los siguientes [datos](https://www.kaggle.com/jacobbaruch/basketball-players-stats-per-season-49-leagues) de Kaggle Datasets, de los que hemos seleccionado solo los de la NBA:

In [2]:
df_nba = pd.read_csv('nba_data.csv',sep=';')
df_nba.head()

,League,Season,Stage,Player,Team,GP,MIN,FGM,FGA,3PM,...,FTA,TOV,PF,ORB,DRB,REB,AST,STL,BLK,PTS
0,NBA,1999 - 2000,Regular_Season,Shaquille O'Neal,LAL,79,3163.0,956,1665,0,...,824,223,255,336,742,1078,299,36,239,2344
1,NBA,1999 - 2000,Regular_Season,Vince Carter,TOR,82,3126.0,788,1696,95,...,551,178,263,150,326,476,322,110,92,2107
2,NBA,1999 - 2000,Regular_Season,Karl Malone,UTA,82,2947.0,752,1476,2,...,739,231,229,169,610,779,304,79,71,2095
3,NBA,1999 - 2000,Regular_Season,Allen Iverson,PHI,70,2853.0,729,1733,89,...,620,230,162,71,196,267,328,144,5,1989
4,NBA,1999 - 2000,Regular_Season,Gary Payton,SEA,82,3425.0,747,1666,177,...,423,224,178,100,429,529,732,153,18,1982


Caracterizamos y comparamos a los jugadores mediante las siguientes cualidades: 

1. Capacidad de anotación 
2. Rebotes.
3. Asistencias.
4. Tapones.
5. Faltas personales. 
    
Para ello usaremos las columnas *PF*, *PTS*, *REB*, *AST* y *BLK*. Como las cifras son totales a lo largo de la temporada, debemos normalizarlas dividiéndolas por en número de partidos

In [3]:
columns = ['PF', 'PTS', 'REB', 'AST', 'BLK']
for col in columns: 
    df_nba[col+'_AVG'] = df_nba[col] / df_nba['GP']

Como disponemos de datos en distintos momentos de la historia, podemos comparar jugadores que alcanzaron su mejor momento en distintas temporadas. 

Por simplicidad, tomamos los datos de la temporada regular y excluimos los *playoffs*.

In [4]:
df_nba = df_nba.set_index(['Player','Season','Stage'])


In [5]:
records = [
    ("Shaquille O'Neal",'1999 - 2000','Regular_Season'), 
    ("LeBron James",'2009 - 2010','Regular_Season'), 
]
sub_df = df_nba.loc[records,:].reset_index()

Para la representación como *radar plot*, necesitamos pasar de formato *wide* a formato *long*, es decir, tenemos que "des-pivotar" la tabla. 

In [6]:
df_1D = sub_df.melt(id_vars=['Player'], value_vars = [col +'_AVG' for col in ['PF', 'PTS', 'REB', 'AST', 'BLK']])

In [7]:
df_1D.head(10)

,Player,variable,value
0,Shaquille O'Neal,PF_AVG,3.227848
1,LeBron James,PF_AVG,1.565789
2,Shaquille O'Neal,PTS_AVG,29.670886
3,LeBron James,PTS_AVG,29.710526
4,Shaquille O'Neal,REB_AVG,13.645570
5,LeBron James,REB_AVG,7.289474
6,Shaquille O'Neal,AST_AVG,3.784810
7,LeBron James,AST_AVG,8.565789
8,Shaquille O'Neal,BLK_AVG,3.025316
9,LeBron James,BLK_AVG,1.013158


Con *plotly.express* solo es posible representar una curva

In [8]:
df_1D[df_1D['Player']=="Shaquille O'Neal"]

,Player,variable,value
0,Shaquille O'Neal,PF_AVG,3.227848
2,Shaquille O'Neal,PTS_AVG,29.670886
4,Shaquille O'Neal,REB_AVG,13.645570
6,Shaquille O'Neal,AST_AVG,3.784810
8,Shaquille O'Neal,BLK_AVG,3.025316


In [9]:
fig = px.line_polar(df_1D[df_1D['Player']=="Shaquille O'Neal"], r="value", theta="variable", line_close=True)
fig.show()

Problema: estamos comparando magnitudes que no son comparables. Necesitamos normalizarlas de modo que todas tenga un rango entre 0 y 1

In [10]:
# Generamos nuevas columnas con nombres más intuitivos
columns_dict = {'PF_AVG': 'FALTAS', 'PTS_AVG': 'PUNTOS', 'REB_AVG': 'REBOTES', 'AST_AVG':'ASISTENCIAS', 'BLK_AVG': 'TAPONES'}
categories = list(columns_dict.values())
# Normalizamos
for (col,new_col) in columns_dict.items():
    df_nba[new_col] = (df_nba[col] - df_nba[col].min())/(df_nba[col].max()-df_nba[col].min())
# Volvemos a quedarnos con los registros que nos interesan
sub_df = df_nba.loc[records,:].reset_index()
df_1D = sub_df.melt(id_vars=['Player'], value_vars = categories)
fig = px.line_polar(df_1D[df_1D['Player']=="Shaquille O'Neal"], r="value", theta="variable", line_close=True)
fig.show()

Si queremos superponer varias curvas debemos usar *Graph Object*

In [11]:
import plotly.graph_objects as go

players = ["Shaquille O'Neal", "LeBron James"]

fig = go.Figure()

for player in players:
    fig.add_trace(go.Scatterpolar(
          r=df_1D.loc[df_1D['Player']==player,'value'],
          mode = 'lines',
          theta=categories,
          fill='toself',
          name=player
    ))



fig.show()

Hacemos un truquito para que las curvas se cierren

In [12]:
fig = go.Figure()

categories_closed = categories + [categories[0]]
for player in players:
    sub_df = df_1D.loc[df_1D['Player']==player,'value']
    sub_df = sub_df.append(sub_df.iloc[:1]) 
    fig.add_trace(go.Scatterpolar(
          r=sub_df,
          mode = 'lines',
          theta=categories_closed,
          fill='toself',
          name=player
    ))

fig.show()